In [1]:
%load_ext autoreload

import os
import sys
from dotenv import load_dotenv

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
if parent_dir not in sys.path:
  sys.path.insert(0, parent_dir)

load_dotenv() #import 

start_year = 2019
end_year = 2024

output_dir = "sec_data"
load_edgar_index = False
load_10k = False
load_13f = False




In [25]:
%autoreload 2

from utils.python_helpers import edgar_service
import pandas as pd
edgar = edgar_service.EdgarService("cmccullough", "cmccullough@microsoft.com")
if load_edgar_index:

  s_and_p = pd.read_csv("sec_data/s&p 500.csv")
  ref = pd.read_csv("sec_data/reference_data.csv")
  cikfilter = s_and_p["CIK"].astype(str)
  cikfilter2 = ref["cik"].astype(str)
  edgar.parse_master_index_file(cik_filter=cikfilter, output_dir="sec_data")
  # edgar.download_filings_for_form("10-K",)

In [ ]:
if load_10k:
  from utils.python_helpers import form_10k_extractor
  extractor = form_10k_extractor.Form10kExtractor()

  extractor.process_directory_files("./sec_data/reference_data.csv", "./sec_data/10k-raw", "./sec_data/10k_extracted")

In [ ]:
%autoreload 2

from utils.python_helpers.form_13f_hr_extractor import Form13F_HR_Extractor
import pandas as pd
download_13F_HR = False
process_13F_HR = True
output_dir = "sec_data"
reference = pd.read_json("sec_data/merged_data.json")

if download_13F_HR:
  cikfilter = reference["cik"].astype(str)
  edgar.download_filings_for_form(form="13F-HR", output_dir=output_dir)

if process_13F_HR:
  extractor = Form13F_HR_Extractor(input_dir=f"{output_dir}/13F-HR", output_dir=output_dir)
  extractor.process_directory_files()


In [67]:
%autoreload 2
import json
import pandas as pd

reference = pd.read_json("sec_data/merged_data.json")
reference = reference.set_index("cik")
display(reference)
# Ensure 'referenceNames' column exists and is initialized as a list if it doesn't exist
if 'referenceNames' not in reference.columns:
    reference['referenceNames'] = [[] for _ in range(len(reference))]

# with open("sec_data/merged_data.json") as f:
#   reference_json = json.load(f)

sec_13f = pd.read_json("sec_data/13f_hr_data.json")
# # sec_13f = sec_13f.set_index("cik")

# for record in reference_json:
#     if record['securities']:
#         cusip_formatted = record['securities'][0]['cusip'].rjust(9, '0')
#         record['cusip6'] = cusip_formatted[:6]
#         # for security in record['securities']:
#         #     sec_13f.loc[sec_13f["cusip6"] == security['cusip6'], "holdings"] += security['holdings']
#     else:
#         record['cusip6'] = ""
# json.dump(reference_json, open("sec_data/merged_data.json", "w"), indent=2)
# for each record in reference, find the corresponding record in sec_13f
# if it exists, add the holdings to the reference data
def add_security_if_not_exists(cik, new_security):
    if cik in reference.index:
        securities = reference.at[cik, 'securities']
        cusips = [security['cusip'] for security in securities]
        if new_security['cusip'] not in cusips:
            securities.append(new_security)
            reference.at[cik, 'securities'] = securities
    else:
        print(f"CIK {cik} not found in reference data.")


with open("sec_data/cik-lookup-data.txt") as f:
  lines = f.readlines()

cik_lookup = {}
for line in lines:
  parts = line.rsplit(':', 2)
  # print(f"{parts[0]}, {parts[1]}")
  if int(parts[1]) in reference.index:    
    cik_lookup[int(parts[1])] = parts[0]

display(cik_lookup)


In [ ]:

from collections import defaultdict

reference = pd.read_json("sec_data/reference_entities.json")
form13d = pd.read_json("sec_data/13d-data.json")

reference.set_index('cik', inplace=True)

def add_security_if_not_exists(cik, new_security):
    if cik in reference.index:
        securities = reference.at[cik, 'securities']
        cusips = [security['cusip'] for security in securities]
        if new_security['cusip'] not in cusips:
            securities.append(new_security)
            reference.at[cik, 'securities'] = securities
    else:
        print(f"CIK {cik} not found in reference data.")
found = []
notfound = []
for index, row in list(reference.iterrows()):
  lookup_value = form13d.loc[form13d["CIK"] == index]['CUSIP'].unique()
  if lookup_value.size > 0:
      found.append(index)
      for cusip in lookup_value:
        security = {"cusip": cusip, "isin": ""}
        add_security_if_not_exists(index, security)
  else:
      notfound.append(index)
  # names.append(row['companyName'].upper().rstrip("."))
  print(lookup_value)
print(len(found))
print(len(notfound))

reference.reset_index(inplace=True)
reference.to_json("sec_data/reference_entities2.json", orient='records', indent=4)
# if 'referenceNames' not in reference.columns:
#     reference['referenceNames'] = [[] for _ in range(len(reference))]


# with open("sec_data/cik-lookup-data.txt") as f:
#   lines = f.readlines()
# print(1067983 in reference.index)
# cik_lookup = defaultdict(list)
# for line in lines:
#   parts = line.rsplit(':', 2)
#   if int(parts[1]) in reference.index: 
#     print(f"{parts[1]}, {parts[0]}")
#     cik_lookup[int(parts[1])].append(parts[0].upper().rstrip("."))

# for index, row in list(reference.iterrows()):
#   names = [value.upper().rstrip(".") for value in reference.at[index, 'referenceNames']]
#   lookup_value = cik_lookup.get(int(index))
#   names.append(row['companyName'].upper().rstrip("."))

#   if lookup_value:
#     print(f"Adding {lookup_value} to {row['companyName']}")
#     names.extend(lookup_value)
#   print(len(names))
#   reference.at[index, 'referenceNames'] = list(set(names))

# reference.reset_index(inplace=True)
# # display(reference)
# reference.to_json("sec_data/reference_entities2.json", orient='records', indent=4)

# for index, row in list(reference.iterrows())[0:10]:
#   print(row['referenceNames'])
# for row in list(reference.itertuples())[0:10]:
#   print(row['companyName'])
# for line in lines:

#   line = line.strip()
#   # [name, cik, empty] = line.split(':')
#   # [empty, name, cik, empty] = line.split(':')

#   parts = line.rsplit(':', 2)
#   name = parts[0]
#   cik = int(parts[1])
#   if not cik in reference.index:
#      print("skipping cik: ", cik)
#      continue
#   print(f"Processing {name} ({cik})")
#   if not isinstance(reference.at[cik, 'referenceNames'], list):
#     reference.at[cik, 'referenceNames'] = []
#   current_name = reference.at[cik, 'companyName'].upper()
#   names = [name.upper(), reference.at[cik, 'companyName'].upper()] + reference.at[cik, 'referenceNames']
#   reference.at[cik, 'referenceNames'] = list(set(names))

#   cusip6 = reference.at[cik, 'cusip6']
#   if cusip6:
#      sec_13_entity_cusips = sec_13f.loc[sec_13f["cusip6"] == cusip6]['cusip'].unique()
#     #  ref_entity_cusips = ref_entity['securities']
#     #  print(ref_entity_cusips)
#     #  sec_13_entity_cusips = sec_13f.loc[sec_13f["cusip6"] == ref_entity['cusip6']]['cusip'].unique()
#     #  add_security_if_not_exists(cik, ref_entity)
#     #  print(sec_13_entity_cusips)
#   else:  
#      sec_13_entity_cusips = sec_13f.loc[sec_13f["companyName"] == name]['cusip'].unique()

#   for cusip in sec_13_entity_cusips:
#     security = {"cusip": cusip, "isin": ""}
#     add_security_if_not_exists(cik, security)
#   # display(reference.loc[cik, 'securities'])
#   # display(reference.loc[cik, 'referenceNames'])
# reference.reset_index()
# reference.to_json("sec_data/reference_entities.json", orient='records', indent=4)
#   cusips = sec_13f.loc[sec_13f["companyName"] == name].value_counts("cusip")
#   print(cusips)

In [ ]:
%autoreload 2
from utils.python_helpers import form_13d_extractor

form_13d_extractor.process_directory("sec_data/13D-raw", "sec_data")


In [ ]:
%autoreload 2

from utils.python_helpers import finance_reports_service
import pandas as pd

company_cik_ticker_ref = pd.read_json("sec_data/sec_company_tickers.json")

finance_reports_service = finance_reports_service.FinanceReportsService("cmcullough", "cmccullough@microsoft.com", company_cik_ticker_ref, "sec_data")

msftInfo = finance_reports_service.get_current_market_data(789019)
msftInfo

In [ ]:
%autoreload 2

from utils.python_helpers import financial_modeling_service


financial_modeling_service.run()

In [11]:
%autoreload 2

from utils.python_helpers.fin_models import subscription_based
import numpy as np
import plotly.express as px
years = np.array([2018, 2019, 2020, 2021, 2022])
# users = np.array([500000, 750000, 1100000, 1600000, 2300000])
users = np.array([2300000,1600000,1100000,750000,500000])

future_years = np.arange(2023, 2028)
scenario = subscription_based.ScenarioAssumptions()

scenario.forecast_exp_growth(years=years, metric=users, forecast_years=future_years)


array([355379.93046613, 244443.25173427, 168136.96609162, 115650.72533574,
        79548.77848452])

In [ ]:
%autoreload 2

from utils.python_helpers.sec_nport_processor import process_nport
process_nport("sec_data/vti_nport.xml", "sec_data/sec_company_tickers.json", "sec_data")

In [ ]:
import csv
import json
from collections import defaultdict


output_file = 'sec_data/merged_data.json'
def merge_data():
    # Read JSON file
    with open(output_file, mode='r') as file:
        json_data = json.load(file)
    # Create a dictionary to store combined data
    combined_data = {}

    for entry in sec_ref_data:
        cik = entry["cik"]
        if cik in combined_data:
            continue
        else:
            tickers = [sec_entry["ticker"] for sec_entry in grouped_sec_data[cik]]
            securities = [{"isin": json_entry["isin"], "cusip": json_entry["cusip"]} for json_entry in grouped_json_data[cik]]
            lei_list = [json_entry["lei"] for json_entry in grouped_json_data[cik] if json_entry["lei"]]
            print((cik, lei_list))
            val = {
                "cik": cik,
                "lei": lei_list[0] if lei_list else "",
                "companyName": entry["companyName"],
                "tickers": tickers,
                "securities": securities
            }
            if cik in csv_data:
                val["cusips"] = csv_data[cik]["cusips"].split(",")

            combined_data[cik] = val
    # Convert dictionary back to JSON array
    output_data = list(combined_data.values())

    # Write output JSON file
    with open(output_file, mode='w') as file:
        json.dump(output_data, file, indent=4)

# Example usage
merge_data()

In [ ]:
from glob import glob
import json
json_files = glob("sec_data\\basic_financials\\*.json")
merged = 'sec_data/merged_data.json'

for json_file in json_files:
    with open(json_file, "r+") as f:
        data = json.load(f)
        if 'cusips' in data:
            value = data['cusips']
            if isinstance(value, list):
                continue
            elif isinstance(value, float):
                value = []
            else:
                value = value.split(",")
            data["cusips"] = value
        else:
            data["cusips"] = []
        f.seek(0)
        json.dump(data, f, indent=4)
        f.truncate()

In [19]:
import json
import pandas as pd
sector_data = pd.read_csv("sector_data/combined2.csv")
reference_data = pd.read_json("sec_data/company_data.json", orient='records')
reference_data.set_index("cik", inplace=True)
found_ticker = []
found_name = []
notfound = []

for index, row in list(reference_data.iterrows()):
  if row["sector"] != None:
    continue
  sector_info = None
  if len(row["tickers"]) > 0:
    sector_info = sector_data.loc[sector_data["Symbol"].isin(row["tickers"])]
    if not sector_info.empty:
      found_ticker.append((index, sector_info["Symbol"], sector_info["Sector"]))
      reference_data.at[index, "sector"] = sector_info.iloc[0]["Sector"]
      reference_data.at[index, "industry"] = sector_info.iloc[0]["Industry"]
      reference_data.at[index, "subindustry"] = sector_info.iloc[0]["Sub-Industry"]
      continue
  if sector_info.empty or sector_info == None:
    sector_info = sector_data.loc[sector_data["Company Name"].str.upper().isin(row["referenceNames"])]
    if not sector_info.empty:
      reference_data.at[index, "sector"] = sector_info.iloc[0]["Sector"]
      reference_data.at[index, "industry"] = sector_info.iloc[0]["Industry"]
      reference_data.at[index, "subindustry"] = sector_info.iloc[0]["Sub-Industry"]
      tickers = reference_data.at[index, "tickers"]
      tickers.extend(sector_info["Symbol"])
      reference_data.at[index, "tickers"] = set(tickers)
      found_name.append((index, sector_info["Symbol"], sector_info["Sector"]))
      continue
  notfound.append(index)
reference_data.reset_index(inplace=True)
reference_data.to_json("sec_data/reference_entities3.json", orient='records', indent=4)
print(len(found_ticker))
print(len(found_name))
print(len(notfound))
print(notfound)